In [11]:
import numpy as np

arr_lvl = list(range(66, 114))
arr_vel = [0.017, 0.035, 0.053, 0.071, 0.088, 0.106, 0.124, 0.142, 0.16, 0.177, 0.195, 0.213, 0.231, 0.248, 0.266, 0.284, 0.302, 0.32, 0.337, 0.355, 0.373, 0.391, 0.408, 0.426, 0.444, 0.462, 0.48, 0.497, 0.515, 0.533, 0.551, 0.568, 0.586, 0.604, 0.622, 0.64, 0.657, 0.675, 0.693, 0.711, 0.728, 0.746, 0.764, 0.782, 0.8, 0.817, 0.835, 0.85]
arr_price = [27700, 39833, 35233, 30700, 44000, 27800, 30400, 31600, 28250, 29400, 37200, 38400, 42000, 55800, 90400, 85000, 88962, 83000, 114500, 247200, 278000, 341333, 388000, 554667, 614167, 623333, 616667, 741667, 965000, 1658333, 2787037, 2772130, 2850000, 3391667, 3951852, 5848333, 5270000, 5166667, 5744444, 6253722, 6150000, 6983333, 7523083, 7584167, 9883333, 10685185, 11703704, 13000000] 

stones = [Stone(l, v, p) for l, v, p in zip(arr_lvl, arr_vel, arr_price)]
stones

class Stone:
    def __init__(self, level, cost, success_prob):
        self.level = int(level)
        self.cost = float(cost)
        self.success_prob = float(success_prob)
        assert 65 <= self.level <= 113, "Уровень камня должен быть между 65 и 113"
        assert 0 <= self.success_prob <= 0.85, "Шанс успеха не может превышать 85%"

# Параметры улучшения/ухудшения
SUCCESS_STEPS = {
    1: 0.90,  # 90% на +1
    2: 0.07,   # 7% на +2
    3: 0.03    # 3% на +3
}

FAIL_STEPS = {
    -1: 0.90,  # 90% на -1
    -2: 0.07,   # 7% на -2
    -3: 0.03    # 3% на -3
}

def calculate_expected_cost(stones):
    MAX_LEVEL = 10
    INIT_VALUE = 1e30  # Большое начальное значение
    TOLERANCE = 1e-6

    E = np.full(MAX_LEVEL + 1, INIT_VALUE, dtype=np.float64)
    best_stones = np.full(MAX_LEVEL + 1, -1, dtype=int)
    E[MAX_LEVEL] = 0.0  # Целевой уровень

    iteration = 0
    converged = False

    while not converged and iteration < 100000:
        converged = True
        max_diff = 0.0

        for current_level in range(MAX_LEVEL-1, -1, -1):
            old_value = E[current_level]
            min_cost = E[current_level]
            best_stone = best_stones[current_level]

            for stone in stones:
                total_expected = 0.0
                cost = stone.cost

                # Рассчитываем успешные улучшения
                success_sum = 0.0
                for step, prob in SUCCESS_STEPS.items():
                    new_level = min(current_level + step, MAX_LEVEL)
                    success_sum += prob * E[new_level]

                # Рассчитываем неудачные ухудшения
                fail_sum = 0.0
                for step, prob in FAIL_STEPS.items():
                    new_level = max(current_level + step, 0)
                    fail_sum += prob * E[new_level]

                # Общая ожидаемая стоимость
                expected = cost + stone.success_prob * success_sum + (1 - stone.success_prob) * fail_sum

                if expected < min_cost:
                    min_cost = expected
                    best_stone = stone.level

            # Проверяем схождение
            diff = abs(old_value - min_cost)
            if diff > TOLERANCE:
                converged = False
                max_diff = max(max_diff, diff)

            E[current_level] = min_cost
            best_stones[current_level] = best_stone

        iteration += 1
        if iteration % 1000 == 0:
            print(f"Iteration {iteration}, Max change: {max_diff:.2f}")

    print(f"\nРасчёт завершён за {iteration} итераций")
    return E, best_stones

# # Пример данных (замените своими)
# stones = [
#     Stone(65, 27700, 0.17),
#     Stone(66, 39833, 0.19),
#     Stone(67, 35233, 0.21),
#     # ... дополнительные камни до уровня 113
#     Stone(113, 150000, 0.85)
# ]

# Проверка данных
for stone in stones:
    assert 65 <= stone.level <= 113, f"Некорректный уровень камня: {stone.level}"
    assert 0 <= stone.success_prob <= 0.85, f"Некорректный шанс для камня {stone.level}"

# Запуск расчёта
try:
    expected_costs, best_stones = calculate_expected_cost(stones)
    
    print("\nРезультаты оптимизации:")
    print("Уровень | Ожидаемая стоимость | Лучший камень")
    print("---------------------------------------------")
    for level in range(11):
        stone_level = best_stones[level]
        cost = expected_costs[level]
        print(f"{level:6} | {cost:19.2f} | {stone_level if stone_level != -1 else '-'}")
        
except Exception as e:
    print(f"Ошибка: {str(e)}")

Iteration 1000, Max change: 1310.17
Iteration 2000, Max change: 23.00
Iteration 3000, Max change: 0.40
Iteration 4000, Max change: 0.01
Iteration 5000, Max change: 0.00
Iteration 6000, Max change: 0.00

Расчёт завершён за 6194 итераций

Результаты оптимизации:
Уровень | Ожидаемая стоимость | Лучший камень
---------------------------------------------
     0 |         35772635.76 | 75
     1 |         35666231.41 | 83
     2 |         35309272.71 | 83
     3 |         34509223.39 | 84
     4 |         32891870.03 | 92
     5 |         30267253.42 | 92
     6 |         26585968.77 | 93
     7 |         21839841.81 | 93
     8 |         16060671.19 | 94
     9 |          9103771.45 | 94
    10 |                0.00 | -


In [15]:
import numpy as np

class Stone:
    def __init__(self, level, cost, success_prob):
        self.level = int(level)
        self.cost = float(cost)
        self.success_prob = float(success_prob)
        assert 65 <= self.level <= 113, "Уровень камня должен быть между 65 и 113"
        assert 0 <= self.success_prob <= 0.85, "Шанс успеха не может превышать 85%"

# Параметры улучшения/ухудшения
SUCCESS_STEPS = {1: 0.90, 2: 0.07, 3: 0.03}
FAIL_STEPS = {-1: 0.90, -2: 0.07, -3: 0.03}

def calculate_expected_cost(stones, top_n=5):
    MAX_LEVEL = 10
    INIT_VALUE = 1e30
    TOLERANCE = 1e-6

    E = np.full(MAX_LEVEL + 1, INIT_VALUE, dtype=np.float64)
    best_options = {i: [] for i in range(MAX_LEVEL + 1)}  # Хранит все варианты
    E[MAX_LEVEL] = 0.0

    # Основной цикл DP
    iteration = 0
    converged = False
    while not converged and iteration < 100000:
        converged = True
        max_diff = 0.0

        for current_level in range(MAX_LEVEL-1, -1, -1):
            old_value = E[current_level]
            min_cost = E[current_level]

            # Рассчитываем стоимость для всех камней
            stone_costs = []
            for stone in stones:
                # Расчет улучшений
                success_sum = 0.0
                for step, prob in SUCCESS_STEPS.items():
                    new_level = min(current_level + step, MAX_LEVEL)
                    success_sum += prob * E[new_level]

                # Расчет ухудшений
                fail_sum = 0.0
                for step, prob in FAIL_STEPS.items():
                    new_level = max(current_level + step, 0)
                    fail_sum += prob * E[new_level]

                # Общая стоимость
                expected = stone.cost + stone.success_prob * success_sum + (1 - stone.success_prob) * fail_sum
                stone_costs.append((stone.level, expected))

                if expected < min_cost:
                    min_cost = expected

            # Обновляем ожидаемую стоимость
            diff = abs(old_value - min_cost)
            if diff > TOLERANCE:
                converged = False
                max_diff = max(max_diff, diff)
            E[current_level] = min_cost

            # Сохраняем все варианты
            sorted_stones = sorted(stone_costs, key=lambda x: x[1])[:top_n]
            best_options[current_level] = sorted_stones

        iteration += 1
        if iteration % 1000 == 0:
            print(f"Iteration {iteration}, Max change: {max_diff:.2f}")

    print(f"\nРасчёт завершён за {iteration} итераций")
    return E, best_options

# Данные
arr_lvl = list(range(66, 114))
arr_vel = [0.017, 0.035, 0.053, 0.071, 0.088, 0.106, 0.124, 0.142, 0.16, 0.177, 
           0.195, 0.213, 0.231, 0.248, 0.266, 0.284, 0.302, 0.32, 0.337, 0.355, 
           0.373, 0.391, 0.408, 0.426, 0.444, 0.462, 0.48, 0.497, 0.515, 0.533, 
           0.551, 0.568, 0.586, 0.604, 0.622, 0.64, 0.657, 0.675, 0.693, 0.711, 
           0.728, 0.746, 0.764, 0.782, 0.8, 0.817, 0.835, 0.85]
arr_price = [27700, 39833, 35233, 30700, 44000, 27800, 30400, 31600, 28250, 29400, 
             37200, 38400, 42000, 55800, 90400, 85000, 88962, 83000, 114500, 247200, 
             278000, 341333, 388000, 554667, 614167, 623333, 616667, 741667, 965000, 
             1658333, 2787037, 2772130, 2850000, 3391667, 3951852, 5848333, 5270000, 
             5166667, 5744444, 6253722, 6150000, 6983333, 7523083, 7584167, 9883333, 
             10685185, 11703704, 13000000]

stones = [Stone(l, p, v) for l, v, p in zip(arr_lvl, arr_vel, arr_price)]

# Проверка данных
for stone in stones:
    assert 65 <= stone.level <= 113, f"Некорректный уровень камня: {stone.level}"
    assert 0 <= stone.success_prob <= 0.85, f"Некорректный шанс для камня {stone.level}"

# Запуск расчёта
try:
    expected_costs, best_options = calculate_expected_cost(stones, top_n=9)
    
    print("\nТоп-5 оптимальных камней для каждого уровня:")
    print("Уровень | Варианты (уровень камня: стоимость)")
    print("---------------------------------------------")
    for level in range(10):
        options = best_options[level]
        print(f"{level+1:6} | ", end="")
        for i, (stone_lvl, cost) in enumerate(options):
            if i > 0: print(" | ", end="")
            print(f"{stone_lvl}: {cost:.0f}", end="")
        print()
        
except Exception as e:
    print(f"Ошибка: {str(e)}")

Iteration 1000, Max change: 1310.17
Iteration 2000, Max change: 23.00
Iteration 3000, Max change: 0.40
Iteration 4000, Max change: 0.01
Iteration 5000, Max change: 0.00
Iteration 6000, Max change: 0.00

Расчёт завершён за 6194 итераций

Топ-5 оптимальных камней для каждого уровня:
Уровень | Варианты (уровень камня: стоимость)
---------------------------------------------
     1 | 75: 35772636 | 74: 35774309 | 77: 35775656 | 78: 35776266 | 76: 35777446 | 73: 35780649 | 72: 35782439 | 71: 35782829 | 79: 35787243
     2 | 83: 35666231 | 78: 35677909 | 79: 35681647 | 82: 35682847 | 77: 35684963 | 84: 35687669 | 81: 35689539 | 76: 35694417 | 75: 35697271
     3 | 83: 35309273 | 84: 35316835 | 82: 35340581 | 81: 35361965 | 79: 35383458 | 80: 35392711 | 78: 35393596 | 77: 35415342 | 85: 35424188
     4 | 84: 34509223 | 83: 34525820 | 86: 34570873 | 88: 34581851 | 82: 34582707 | 87: 34583280 | 85: 34590998 | 92: 34606816 | 81: 34629671
     5 | 92: 32891870 | 93: 32934536 | 91: 32985714 | 88: 